In [2]:
#All the import statements goes here
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import matplotlib as plt
import os
import pickle

path1 = "J:/SF/Thermal Load Prediction/Repos folders/loadPrediction/"
path_list = ["J:/SF/Thermal Load Prediction/Datas/Open Power System Data/opsd-household_data-2017-09-01/",
             "/Volumes/ADITHYA/Adithya/ML materials/DATA SETS/Open Power System /opsd-household_data-2017-09-01/"]
path_sd = ["/media/siddharth/New Volume/Github/TherML Data/Open Power System Data/opsd-household_data-2017-09-01/"]
path_sfroot = ["/home/sfroot/loadPrediction/"]

if("adithya" in os.getlogin()):
    path = path_list[1]
elif("siddharth" in os.getlogin()):
    path = path_sd[0]
elif("sfroot" in os.getlogin()):    
    path = path_sfroot[0]
else:
    path=path1

In [3]:
# read1 contains all rows and columns from csv file
read1 = pd.read_csv(path+"household_data_1min_singleindex.csv")

# read will have columns with load data
read = read1[read1.columns[2:11]]
x = np.array(read.values)

# t is to find out which minute in the data has minimum occurances of NaN
t = np.zeros(shape=(60))

for i in range(60):
    j=i;
    c=0
    while(j <= x.shape[0]-1):
        c1=0
        for k in x[j]:
            if(np.isnan(k)):
                c1+=1
        if(c1!=0):
            c+=1
        j+=60
    t[i]=c
    
# amin(t) gives minimum count and argmin(t)+1 gives minute value
print(np.amin(t))
print(np.argmin(t)+1)

FileNotFoundError: File b'/home/sfroot/loadPrediction/household_data_1min_singleindex.csv' does not exist

In [4]:
# using 1hr data
hr_data = pd.read_csv(path+"household_data_60min_singleindex.csv")
# removing the following columns
del hr_data["interpolated_values"]
del hr_data["DE_KN_residential4_grid_export"]
del hr_data["DE_KN_residential4_pv"]
del hr_data["cet_cest_timestamp"]

# yr_data will have the data from 2016-01-01 to 2017-01-01
yr_data = pd.DataFrame(hr_data.values[2137:10921])
yr_data.columns = hr_data.columns

In [5]:
yr_data.head()

,utc_timestamp,DE_KN_residential4_dishwasher,DE_KN_residential4_ev,DE_KN_residential4_freezer,DE_KN_residential4_grid_import,DE_KN_residential4_heat_pump,DE_KN_residential4_refrigerator,DE_KN_residential4_washing_machine
0,2016-01-01T02:00:00Z,25.092,203.147,35.935,1146.79,798.665,29.125,16.217
1,2016-01-01T03:00:00Z,25.092,203.147,35.949,1147.79,799.32,29.125,16.217
2,2016-01-01T04:00:00Z,25.092,203.147,35.963,1148.54,799.855,29.125,16.217
3,2016-01-01T05:00:00Z,25.092,203.147,35.982,1149.23,800.335,29.172,16.217
4,2016-01-01T06:00:00Z,25.092,203.147,36.002,1150.04,800.995,29.172,16.217


In [6]:
# doing linear extrapolation on data ..ie.. if x is notNull and y in notNull and we have a set of nulls between them,
# we do : 
# gradient = (y-x)/(no. of nulls)
# add gradient to values from x to y recursively so that we fill the gap
df = pd.DataFrame()
for i in range(len(yr_data.columns)):
    if(i != 0):
        df = yr_data.iloc[:,i]
        flag1 = False
        for j in range(df.values.shape[0]):
            if(np.isnan(df.values[j]) and not(flag1)):
                prev = j-1
                flag1 = True
            elif(flag1 and not(np.isnan(df.values[j]))):
                grad = (df.values[j] - df.values[prev])/(j-prev)
                for k in range(prev+1, j):
                    df.values[k] = df.values[k-1] + grad
                flag1 = False
        yr_data[yr_data.columns[i]] = df.values

In [7]:
yr_data.isnull().any()

utc_timestamp                         False
DE_KN_residential4_dishwasher         False
DE_KN_residential4_ev                 False
DE_KN_residential4_freezer            False
DE_KN_residential4_grid_import        False
DE_KN_residential4_heat_pump          False
DE_KN_residential4_refrigerator       False
DE_KN_residential4_washing_machine    False
dtype: bool

In [21]:
# exporting the new non null containing data to new csv
yr_data.to_csv("Year_Data_New.csv",sep=",",encoding="utf-8")

In [22]:
1.0*381/np.sum(hr_data.isnull().sum())

0.08397619572404673

In [8]:
hr_data.isnull().sum()

utc_timestamp                            0
DE_KN_residential4_dishwasher         1083
DE_KN_residential4_ev                  405
DE_KN_residential4_freezer             793
DE_KN_residential4_grid_import         343
DE_KN_residential4_heat_pump           298
DE_KN_residential4_refrigerator       1234
DE_KN_residential4_washing_machine     381
dtype: int64

In [9]:
hr_data.shape

(11867, 8)

In [10]:
# doing linear extrapolation on data ..ie.. if x is notNull and y in notNull and we have a set of nulls between them,
# we do : 
# gradient = (y-x)/(no. of nulls)
# add gradient to values from x to y recursively so that we fill the gap on hr data
df = pd.DataFrame()
for i in range(len(hr_data.columns)):
    if(i != 0):
        df = hr_data.iloc[:,i]
        flag1 = False
        flag2 = False
        for j in range(df.values.shape[0]):
            if(np.isnan(df.values[j]) and not(flag1)):
                if(j==0):
                    flag2 = True
                prev = j-1
                flag1 = True
            elif(flag1 and not(np.isnan(df.values[j]))):
                if(flag2):
                    flag2 = False
                    for k in range(prev+1,j):
                        df.values[k] = df.values[j]
                else:
                    grad = (df.values[j] - df.values[prev])/(j-prev)
                    for k in range(prev+1, j):
                        df.values[k] = df.values[k-1] + grad
                flag1 = False
        hr_data[hr_data.columns[i]] = df.values

In [11]:
hr_data.shape

(11867, 8)

In [12]:
hr_data.head()

,utc_timestamp,DE_KN_residential4_dishwasher,DE_KN_residential4_ev,DE_KN_residential4_freezer,DE_KN_residential4_grid_import,DE_KN_residential4_heat_pump,DE_KN_residential4_refrigerator,DE_KN_residential4_washing_machine
0,2015-10-04T01:00:00Z,0.002,0.945,0.012,0.077,0.00,0.0,0.001
1,2015-10-04T02:00:00Z,0.002,0.945,0.012,0.077,0.02,0.0,0.001
2,2015-10-04T03:00:00Z,0.002,0.945,0.012,0.077,0.03,0.0,0.001
3,2015-10-04T04:00:00Z,0.002,0.945,0.012,0.077,0.04,0.0,0.001
4,2015-10-04T05:00:00Z,0.002,0.945,0.012,0.077,0.06,0.0,0.001


In [13]:
# counted null values after interpolation in entire dataset

hr_data = hr_data.iloc[:11012,:]

nan_cnt = []

nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_dishwasher'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_ev'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_freezer'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_grid_import'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_heat_pump'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_refrigerator'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_washing_machine'])))

print(nan_cnt)

[0, 0, 0, 0, 0, 0, 0]


In [14]:
#Another efficient way to check if there are any NULLS present in any of the columns.
hr_data.isnull().sum(axis=0)

utc_timestamp                         0
DE_KN_residential4_dishwasher         0
DE_KN_residential4_ev                 0
DE_KN_residential4_freezer            0
DE_KN_residential4_grid_import        0
DE_KN_residential4_heat_pump          0
DE_KN_residential4_refrigerator       0
DE_KN_residential4_washing_machine    0
dtype: int64

In [38]:
hr_data.to_csv("Hour_Data_New.csv",sep=",",encoding="utf-8")
#hr_data = pd.read_csv(path1+"Hour_Data_New.csv")
#del hr_data['Unnamed: 0']

In [136]:
'''
Function that takes in a dataframe and then returns the dictionary of accuracies.
'''

def nn_model(localdf,localdict,times):
    for counter in xrange(times):
        for i in range(len(localdf.columns)):
            df = pd.DataFrame()
            if(i != 0):
                print(localdf.columns[i])
                df = localdf[localdf.columns[i]]
                ndf = pd.concat([df.shift(2), df.shift(1), df], axis=1)
                ndf.columns = ('t-2', 't-1', 't')
                ndf = ndf.iloc[2:]

                x = ndf[ndf.columns[0:2]]
                y = ndf[ndf.columns[-1:]]

                X_train,X_test,y_train,y_test = train_test_split(x.values,y.values,test_size=0.2,random_state=1)
                model = Sequential()
                model.add(Dense(5,input_dim=2,kernel_initializer='normal', activation='relu'))
                #Output Layer.
                model.add(Dense(1,kernel_initializer='normal'))
                #Compile model.
                model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mean_squared_error'])
                #Fit the model.
                model.fit(X_train,y_train,epochs=200,batch_size=200,verbose=2)
                #Final Evaluation of the model.
                scores = model.evaluate(X_test,y_test,verbose=0)
                print ("Baseline Error : %.5f" % (scores[1]))
                localdict.setdefault(counter,[]).append((scores[1]))
                model.summary()
    return localdict            

In [67]:
'''
Function that computes the actual load consumed, rather the meter readings.
'''
def actual_load(df):
    data = []
    temp = np.zeros((1,7))
    tempdate = []
    
    print (df.shape)
    
    for i in range(df.shape[0]-1,1,-1):
        if((i%1000)==0):
            print (i)
        temp = df.values[i,1:] - df.values[i-1,1:]  
        tempdate.append(df.values[i,0])
        data.append(temp)
    
    tempdate = np.array(tempdate)
    return tempdate,data        

In [112]:
hr_data_copy = hr_data
tempdate,data = actual_load(hr_data_copy)

(11012, 8)
11000
10000
9000
8000
7000
6000
5000
4000
3000
2000
1000


In [113]:
#To chage the dimension in order to perform hstack
tempdate = tempdate.reshape((tempdate.shape[0],-1))
data = np.hstack((tempdate,data))
#To reverse the array
data = data[::-1]
np.shape(data)

(11010, 8)

In [121]:
new_df = pd.DataFrame(data,columns=hr_data.columns)
print (new_df.shape,hr_data.shape)

(11010, 8) (11012, 8)


In [117]:
hr_data.tail()

,utc_timestamp,DE_KN_residential4_dishwasher,DE_KN_residential4_ev,DE_KN_residential4_freezer,DE_KN_residential4_grid_import,DE_KN_residential4_heat_pump,DE_KN_residential4_refrigerator,DE_KN_residential4_washing_machine
11007,2017-01-04T16:00:00Z,117.198,1229.077,199.749,5177.600,3477.680,191.874,134.062
11008,2017-01-04T17:00:00Z,117.199,1229.077,199.757,5178.388,3478.287,191.886,134.062
11009,2017-01-04T18:00:00Z,117.199,1229.077,199.766,5179.642,3479.360,191.912,134.062
11010,2017-01-04T19:00:00Z,117.199,1229.077,199.781,5180.477,3479.945,191.912,134.062
11011,2017-01-04T20:00:00Z,117.199,1229.077,199.796,5181.974,3481.030,191.951,134.062


In [118]:
new_df.tail()

,utc_timestamp,DE_KN_residential4_dishwasher,DE_KN_residential4_ev,DE_KN_residential4_freezer,DE_KN_residential4_grid_import,DE_KN_residential4_heat_pump,DE_KN_residential4_refrigerator,DE_KN_residential4_washing_machine
11005,2017-01-04T16:00:00Z,0.74,0.002,0.015,1.9,1.003,0,0.003
11006,2017-01-04T17:00:00Z,0.001,0,0.008,0.788,0.607,0.012,0
11007,2017-01-04T18:00:00Z,0,0,0.009,1.254,1.073,0.026,0
11008,2017-01-04T19:00:00Z,0,0,0.015,0.835,0.585,0,0
11009,2017-01-04T20:00:00Z,0,0,0.015,1.497,1.085,0.039,0


In [23]:
# relu activation for input layer alone.. output layer softmax made it worser..
# this is on the year 2016-01-01 to 2017-01-01 data

#acc_dict1={}

acc={}
acc = nn_model(yr_data,acc,1)
print acc

DE_KN_residential4_dishwasher
Epoch 1/200
0s - loss: 5712.9227 - mean_squared_error: 5712.9227
Epoch 2/200
0s - loss: 5580.0774 - mean_squared_error: 5580.0774
Epoch 3/200
0s - loss: 5324.4503 - mean_squared_error: 5324.4503
Epoch 4/200
0s - loss: 4937.2259 - mean_squared_error: 4937.2259
Epoch 5/200
0s - loss: 4434.2120 - mean_squared_error: 4434.2120
Epoch 6/200
0s - loss: 3848.1928 - mean_squared_error: 3848.1928
Epoch 7/200
0s - loss: 3217.9019 - mean_squared_error: 3217.9019
Epoch 8/200
0s - loss: 2583.7028 - mean_squared_error: 2583.7028
Epoch 9/200
0s - loss: 1982.4029 - mean_squared_error: 1982.4029
Epoch 10/200
0s - loss: 1445.6938 - mean_squared_error: 1445.6938
Epoch 11/200
0s - loss: 998.5292 - mean_squared_error: 998.5292
Epoch 12/200
0s - loss: 648.9525 - mean_squared_error: 648.9525
Epoch 13/200
0s - loss: 395.5523 - mean_squared_error: 395.5523
Epoch 14/200
0s - loss: 226.4543 - mean_squared_error: 226.4543
Epoch 15/200
0s - loss: 120.5447 - mean_squared_error: 120.5447

0s - loss: 0.0659 - mean_squared_error: 0.0659
Epoch 137/200
0s - loss: 0.0656 - mean_squared_error: 0.0656
Epoch 138/200
0s - loss: 0.0653 - mean_squared_error: 0.0653
Epoch 139/200
0s - loss: 0.0650 - mean_squared_error: 0.0650
Epoch 140/200
0s - loss: 0.0647 - mean_squared_error: 0.0647
Epoch 141/200
0s - loss: 0.0643 - mean_squared_error: 0.0643
Epoch 142/200
0s - loss: 0.0640 - mean_squared_error: 0.0640
Epoch 143/200
0s - loss: 0.0636 - mean_squared_error: 0.0636
Epoch 144/200
0s - loss: 0.0633 - mean_squared_error: 0.0633
Epoch 145/200
0s - loss: 0.0629 - mean_squared_error: 0.0629
Epoch 146/200
0s - loss: 0.0626 - mean_squared_error: 0.0626
Epoch 147/200
0s - loss: 0.0622 - mean_squared_error: 0.0622
Epoch 148/200
0s - loss: 0.0618 - mean_squared_error: 0.0618
Epoch 149/200
0s - loss: 0.0615 - mean_squared_error: 0.0615
Epoch 150/200
0s - loss: 0.0611 - mean_squared_error: 0.0611
Epoch 151/200
0s - loss: 0.0607 - mean_squared_error: 0.0607
Epoch 152/200
0s - loss: 0.0603 - mean

0s - loss: 0.3814 - mean_squared_error: 0.3814
Epoch 64/200
0s - loss: 0.3814 - mean_squared_error: 0.3814
Epoch 65/200
0s - loss: 0.3814 - mean_squared_error: 0.3814
Epoch 66/200
0s - loss: 0.3814 - mean_squared_error: 0.3814
Epoch 67/200
0s - loss: 0.3813 - mean_squared_error: 0.3813
Epoch 68/200
0s - loss: 0.3814 - mean_squared_error: 0.3814
Epoch 69/200
0s - loss: 0.3813 - mean_squared_error: 0.3813
Epoch 70/200
0s - loss: 0.3814 - mean_squared_error: 0.3814
Epoch 71/200
0s - loss: 0.3814 - mean_squared_error: 0.3814
Epoch 72/200
0s - loss: 0.3812 - mean_squared_error: 0.3812
Epoch 73/200
0s - loss: 0.3812 - mean_squared_error: 0.3812
Epoch 74/200
0s - loss: 0.3813 - mean_squared_error: 0.3813
Epoch 75/200
0s - loss: 0.3812 - mean_squared_error: 0.3812
Epoch 76/200
0s - loss: 0.3812 - mean_squared_error: 0.3812
Epoch 77/200
0s - loss: 0.3812 - mean_squared_error: 0.3812
Epoch 78/200
0s - loss: 0.3811 - mean_squared_error: 0.3811
Epoch 79/200
0s - loss: 0.3811 - mean_squared_error: 

Baseline Error : 63.53173
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 5)                 15        
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 6         
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________
DE_KN_residential4_freezer
Epoch 1/200
0s - loss: 15789.0255 - mean_squared_error: 15789.0255
Epoch 2/200
0s - loss: 14556.9890 - mean_squared_error: 14556.9890
Epoch 3/200
0s - loss: 12505.8816 - mean_squared_error: 12505.8816
Epoch 4/200
0s - loss: 9796.7895 - mean_squared_error: 9796.7895
Epoch 5/200
0s - loss: 6838.9810 - mean_squared_error: 6838.9810
Epoch 6/200
0s - loss: 4155.0390 - mean_squared_error: 4155.0390
Epoch 7/200
0s - loss: 2126.6670 - mean_squared_error: 2126.6670
Epoch 8/200
0s - loss: 892.2

0s - loss: 0.0452 - mean_squared_error: 0.0452
Epoch 127/200
0s - loss: 0.0448 - mean_squared_error: 0.0448
Epoch 128/200
0s - loss: 0.0444 - mean_squared_error: 0.0444
Epoch 129/200
0s - loss: 0.0440 - mean_squared_error: 0.0440
Epoch 130/200
0s - loss: 0.0436 - mean_squared_error: 0.0436
Epoch 131/200
0s - loss: 0.0432 - mean_squared_error: 0.0432
Epoch 132/200
0s - loss: 0.0428 - mean_squared_error: 0.0428
Epoch 133/200
0s - loss: 0.0424 - mean_squared_error: 0.0424
Epoch 134/200
0s - loss: 0.0420 - mean_squared_error: 0.0420
Epoch 135/200
0s - loss: 0.0415 - mean_squared_error: 0.0415
Epoch 136/200
0s - loss: 0.0411 - mean_squared_error: 0.0411
Epoch 137/200
0s - loss: 0.0407 - mean_squared_error: 0.0407
Epoch 138/200
0s - loss: 0.0403 - mean_squared_error: 0.0403
Epoch 139/200
0s - loss: 0.0399 - mean_squared_error: 0.0399
Epoch 140/200
0s - loss: 0.0395 - mean_squared_error: 0.0395
Epoch 141/200
0s - loss: 0.0390 - mean_squared_error: 0.0390
Epoch 142/200
0s - loss: 0.0386 - mean

...

0s - loss: 0.5205 - mean_squared_error: 0.5205
Epoch 188/200
0s - loss: 0.5208 - mean_squared_error: 0.5208
Epoch 189/200
0s - loss: 0.5203 - mean_squared_error: 0.5203
Epoch 190/200
0s - loss: 0.5206 - mean_squared_error: 0.5206
Epoch 191/200
0s - loss: 0.5206 - mean_squared_error: 0.5206
Epoch 192/200
0s - loss: 0.5204 - mean_squared_error: 0.5204
Epoch 193/200
0s - loss: 0.5207 - mean_squared_error: 0.5207
Epoch 194/200
0s - loss: 0.5202 - mean_squared_error: 0.5202
Epoch 195/200
0s - loss: 0.5224 - mean_squared_error: 0.5224
Epoch 196/200
0s - loss: 0.5209 - mean_squared_error: 0.5209
Epoch 197/200
0s - loss: 0.5221 - mean_squared_error: 0.5221
Epoch 198/200
0s - loss: 0.5218 - mean_squared_error: 0.5218
Epoch 199/200
0s - loss: 0.5205 - mean_squared_error: 0.5205
Epoch 200/200
0s - loss: 0.5207 - mean_squared_error: 0.5207
Baseline Error : 45.82523
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
de

0s - loss: 0.3043 - mean_squared_error: 0.3043
Epoch 115/200
0s - loss: 0.3045 - mean_squared_error: 0.3045
Epoch 116/200
0s - loss: 0.3043 - mean_squared_error: 0.3043
Epoch 117/200
0s - loss: 0.3041 - mean_squared_error: 0.3041
Epoch 118/200
0s - loss: 0.3044 - mean_squared_error: 0.3044
Epoch 119/200
0s - loss: 0.3040 - mean_squared_error: 0.3040
Epoch 120/200
0s - loss: 0.3043 - mean_squared_error: 0.3043
Epoch 121/200
0s - loss: 0.3043 - mean_squared_error: 0.3043
Epoch 122/200
0s - loss: 0.3041 - mean_squared_error: 0.3041
Epoch 123/200
0s - loss: 0.3040 - mean_squared_error: 0.3040
Epoch 124/200
0s - loss: 0.3040 - mean_squared_error: 0.3040
Epoch 125/200
0s - loss: 0.3048 - mean_squared_error: 0.3048
Epoch 126/200
0s - loss: 0.3043 - mean_squared_error: 0.3043
Epoch 127/200
0s - loss: 0.3041 - mean_squared_error: 0.3041
Epoch 128/200
0s - loss: 0.3040 - mean_squared_error: 0.3040
Epoch 129/200
0s - loss: 0.3045 - mean_squared_error: 0.3045
Epoch 130/200
0s - loss: 0.3045 - mean

0s - loss: 0.0962 - mean_squared_error: 0.0962
Epoch 39/200
0s - loss: 0.0961 - mean_squared_error: 0.0961
Epoch 40/200
0s - loss: 0.0960 - mean_squared_error: 0.0960
Epoch 41/200
0s - loss: 0.0958 - mean_squared_error: 0.0958
Epoch 42/200
0s - loss: 0.0957 - mean_squared_error: 0.0957
Epoch 43/200
0s - loss: 0.0955 - mean_squared_error: 0.0955
Epoch 44/200
0s - loss: 0.0953 - mean_squared_error: 0.0953
Epoch 45/200
0s - loss: 0.0952 - mean_squared_error: 0.0952
Epoch 46/200
0s - loss: 0.0950 - mean_squared_error: 0.0950
Epoch 47/200
0s - loss: 0.0948 - mean_squared_error: 0.0948
Epoch 48/200
0s - loss: 0.0947 - mean_squared_error: 0.0947
Epoch 49/200
0s - loss: 0.0945 - mean_squared_error: 0.0945
Epoch 50/200
0s - loss: 0.0943 - mean_squared_error: 0.0943
Epoch 51/200
0s - loss: 0.0941 - mean_squared_error: 0.0941
Epoch 52/200
0s - loss: 0.0939 - mean_squared_error: 0.0939
Epoch 53/200
0s - loss: 0.0937 - mean_squared_error: 0.0937
Epoch 54/200
0s - loss: 0.0935 - mean_squared_error: 

0s - loss: 0.0328 - mean_squared_error: 0.0328
Epoch 179/200
0s - loss: 0.0321 - mean_squared_error: 0.0321
Epoch 180/200
0s - loss: 0.0315 - mean_squared_error: 0.0315
Epoch 181/200
0s - loss: 0.0307 - mean_squared_error: 0.0307
Epoch 182/200
0s - loss: 0.0300 - mean_squared_error: 0.0300
Epoch 183/200
0s - loss: 0.0293 - mean_squared_error: 0.0293
Epoch 184/200
0s - loss: 0.0286 - mean_squared_error: 0.0286
Epoch 185/200
0s - loss: 0.0280 - mean_squared_error: 0.0280
Epoch 186/200
0s - loss: 0.0273 - mean_squared_error: 0.0273
Epoch 187/200
0s - loss: 0.0266 - mean_squared_error: 0.0266
Epoch 188/200
0s - loss: 0.0260 - mean_squared_error: 0.0260
Epoch 189/200
0s - loss: 0.0253 - mean_squared_error: 0.0253
Epoch 190/200
0s - loss: 0.0247 - mean_squared_error: 0.0247
Epoch 191/200
0s - loss: 0.0240 - mean_squared_error: 0.0240
Epoch 192/200
0s - loss: 0.0234 - mean_squared_error: 0.0234
Epoch 193/200
0s - loss: 0.0228 - mean_squared_error: 0.0228
Epoch 194/200
0s - loss: 0.0221 - mean

0s - loss: 6824.2699 - mean_squared_error: 6824.2699
Epoch 99/200
0s - loss: 6818.8936 - mean_squared_error: 6818.8936
Epoch 100/200
0s - loss: 6813.5333 - mean_squared_error: 6813.5333
Epoch 101/200
0s - loss: 6808.1690 - mean_squared_error: 6808.1690
Epoch 102/200
0s - loss: 6802.8013 - mean_squared_error: 6802.8013
Epoch 103/200
0s - loss: 6797.4587 - mean_squared_error: 6797.4587
Epoch 104/200
0s - loss: 6792.1118 - mean_squared_error: 6792.1118
Epoch 105/200
0s - loss: 6786.7810 - mean_squared_error: 6786.7810
Epoch 106/200
0s - loss: 6781.4227 - mean_squared_error: 6781.4227
Epoch 107/200
0s - loss: 6776.0657 - mean_squared_error: 6776.0657
Epoch 108/200
0s - loss: 6770.7241 - mean_squared_error: 6770.7241
Epoch 109/200
0s - loss: 6765.3835 - mean_squared_error: 6765.3835
Epoch 110/200
0s - loss: 6760.0460 - mean_squared_error: 6760.0460
Epoch 111/200
0s - loss: 6754.7146 - mean_squared_error: 6754.7146
Epoch 112/200
0s - loss: 6749.3707 - mean_squared_error: 6749.3707
Epoch 113/

In [44]:
hr_data.columns

Index([u'utc_timestamp', u'DE_KN_residential4_dishwasher',
       u'DE_KN_residential4_ev', u'DE_KN_residential4_freezer',
       u'DE_KN_residential4_grid_import', u'DE_KN_residential4_heat_pump',
       u'DE_KN_residential4_refrigerator',
       u'DE_KN_residential4_washing_machine'],
      dtype='object')

In [21]:
# relu activation for input layer alone.. output layer softmax made it worser..
# this is on entire dataset till first row with null values after interpolation

#acc_dict={}

acc_1 ={}
acc_1 = nn_model(hr_data,acc_1,1)
print (acc_1)

DE_KN_residential4_dishwasher
Epoch 1/200
0s - loss: 4682.6052 - mean_squared_error: 4682.6052
Epoch 2/200
0s - loss: 4366.0197 - mean_squared_error: 4366.0197
Epoch 3/200
0s - loss: 3754.1313 - mean_squared_error: 3754.1313
Epoch 4/200
0s - loss: 2904.7404 - mean_squared_error: 2904.7404
Epoch 5/200
0s - loss: 1986.0098 - mean_squared_error: 1986.0098
Epoch 6/200
0s - loss: 1166.5387 - mean_squared_error: 1166.5387
Epoch 7/200
0s - loss: 573.0625 - mean_squared_error: 573.0625
Epoch 8/200
0s - loss: 225.7630 - mean_squared_error: 225.7630
Epoch 9/200
0s - loss: 70.5051 - mean_squared_error: 70.5051
Epoch 10/200
0s - loss: 17.8413 - mean_squared_error: 17.8413
Epoch 11/200
0s - loss: 3.7187 - mean_squared_error: 3.7187
Epoch 12/200
0s - loss: 0.7264 - mean_squared_error: 0.7264
Epoch 13/200
0s - loss: 0.2070 - mean_squared_error: 0.2070
Epoch 14/200
0s - loss: 0.1332 - mean_squared_error: 0.1332
Epoch 15/200
0s - loss: 0.1238 - mean_squared_error: 0.1238
Epoch 16/200
0s - loss: 0.1225 

0s - loss: 0.0207 - mean_squared_error: 0.0207
Epoch 139/200
0s - loss: 0.0200 - mean_squared_error: 0.0200
Epoch 140/200
0s - loss: 0.0194 - mean_squared_error: 0.0194
Epoch 141/200
0s - loss: 0.0187 - mean_squared_error: 0.0187
Epoch 142/200
0s - loss: 0.0181 - mean_squared_error: 0.0181
Epoch 143/200
0s - loss: 0.0176 - mean_squared_error: 0.0176
Epoch 144/200
0s - loss: 0.0169 - mean_squared_error: 0.0169
Epoch 145/200
0s - loss: 0.0164 - mean_squared_error: 0.0164
Epoch 146/200
0s - loss: 0.0159 - mean_squared_error: 0.0159
Epoch 147/200
0s - loss: 0.0153 - mean_squared_error: 0.0153
Epoch 148/200
0s - loss: 0.0148 - mean_squared_error: 0.0148
Epoch 149/200
0s - loss: 0.0144 - mean_squared_error: 0.0144
Epoch 150/200
0s - loss: 0.0139 - mean_squared_error: 0.0139
Epoch 151/200
0s - loss: 0.0135 - mean_squared_error: 0.0135
Epoch 152/200
0s - loss: 0.0131 - mean_squared_error: 0.0131
Epoch 153/200
0s - loss: 0.0127 - mean_squared_error: 0.0127
Epoch 154/200
0s - loss: 0.0123 - mean

0s - loss: 0.4145 - mean_squared_error: 0.4145
Epoch 64/200
0s - loss: 0.4148 - mean_squared_error: 0.4148
Epoch 65/200
0s - loss: 0.4143 - mean_squared_error: 0.4143
Epoch 66/200
0s - loss: 0.4141 - mean_squared_error: 0.4141
Epoch 67/200
0s - loss: 0.4140 - mean_squared_error: 0.4140
Epoch 68/200
0s - loss: 0.4139 - mean_squared_error: 0.4139
Epoch 69/200
0s - loss: 0.4137 - mean_squared_error: 0.4137
Epoch 70/200
0s - loss: 0.4138 - mean_squared_error: 0.4138
Epoch 71/200
0s - loss: 0.4135 - mean_squared_error: 0.4135
Epoch 72/200
0s - loss: 0.4133 - mean_squared_error: 0.4133
Epoch 73/200
0s - loss: 0.4135 - mean_squared_error: 0.4135
Epoch 74/200
0s - loss: 0.4132 - mean_squared_error: 0.4132
Epoch 75/200
0s - loss: 0.4129 - mean_squared_error: 0.4129
Epoch 76/200
0s - loss: 0.4128 - mean_squared_error: 0.4128
Epoch 77/200
0s - loss: 0.4126 - mean_squared_error: 0.4126
Epoch 78/200
0s - loss: 0.4126 - mean_squared_error: 0.4126
Epoch 79/200
0s - loss: 0.4131 - mean_squared_error: 

0s - loss: 0.3501 - mean_squared_error: 0.3501
Epoch 199/200
0s - loss: 0.3490 - mean_squared_error: 0.3490
Epoch 200/200
0s - loss: 0.3485 - mean_squared_error: 0.3485
Baseline Error : 67.56452
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 5)                 15        
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 6         
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________
DE_KN_residential4_freezer
Epoch 1/200
0s - loss: 13278.0791 - mean_squared_error: 13278.0791
Epoch 2/200
0s - loss: 12531.8717 - mean_squared_error: 12531.8717
Epoch 3/200
0s - loss: 10919.6146 - mean_squared_error: 10919.6146
Epoch 4/200
0s - loss: 8535.7610 - mean_squared_error: 8535.7610
Epoch 5/200
0s - loss: 5857.0897 - mean_squared_error:

0s - loss: 0.0425 - mean_squared_error: 0.0425
Epoch 126/200
0s - loss: 0.0412 - mean_squared_error: 0.0412
Epoch 127/200
0s - loss: 0.0400 - mean_squared_error: 0.0400
Epoch 128/200
0s - loss: 0.0387 - mean_squared_error: 0.0387
Epoch 129/200
0s - loss: 0.0374 - mean_squared_error: 0.0374
Epoch 130/200
0s - loss: 0.0362 - mean_squared_error: 0.0362
Epoch 131/200
0s - loss: 0.0350 - mean_squared_error: 0.0350
Epoch 132/200
0s - loss: 0.0338 - mean_squared_error: 0.0338
Epoch 133/200
0s - loss: 0.0326 - mean_squared_error: 0.0326
Epoch 134/200
0s - loss: 0.0314 - mean_squared_error: 0.0314
Epoch 135/200
0s - loss: 0.0302 - mean_squared_error: 0.0302
Epoch 136/200
0s - loss: 0.0291 - mean_squared_error: 0.0291
Epoch 137/200
0s - loss: 0.0280 - mean_squared_error: 0.0280
Epoch 138/200
0s - loss: 0.0269 - mean_squared_error: 0.0269
Epoch 139/200
0s - loss: 0.0258 - mean_squared_error: 0.0258
Epoch 140/200
0s - loss: 0.0247 - mean_squared_error: 0.0247
Epoch 141/200
0s - loss: 0.0237 - mean

...

0s - loss: 0.3025 - mean_squared_error: 0.3025
Epoch 71/200
0s - loss: 0.3025 - mean_squared_error: 0.3025
Epoch 72/200
0s - loss: 0.3025 - mean_squared_error: 0.3025
Epoch 73/200
0s - loss: 0.3026 - mean_squared_error: 0.3026
Epoch 74/200
0s - loss: 0.3024 - mean_squared_error: 0.3024
Epoch 75/200
0s - loss: 0.3023 - mean_squared_error: 0.3023
Epoch 76/200
0s - loss: 0.3027 - mean_squared_error: 0.3027
Epoch 77/200
0s - loss: 0.3024 - mean_squared_error: 0.3024
Epoch 78/200
0s - loss: 0.3025 - mean_squared_error: 0.3025
Epoch 79/200
0s - loss: 0.3025 - mean_squared_error: 0.3025
Epoch 80/200
0s - loss: 0.3027 - mean_squared_error: 0.3027
Epoch 81/200
0s - loss: 0.3023 - mean_squared_error: 0.3023
Epoch 82/200
0s - loss: 0.3025 - mean_squared_error: 0.3025
Epoch 83/200
0s - loss: 0.3026 - mean_squared_error: 0.3026
Epoch 84/200
0s - loss: 0.3030 - mean_squared_error: 0.3030
Epoch 85/200
0s - loss: 0.3026 - mean_squared_error: 0.3026
Epoch 86/200
0s - loss: 0.3027 - mean_squared_error: 

Epoch 1/200
0s - loss: 11198.7672 - mean_squared_error: 11198.7672
Epoch 2/200
0s - loss: 10100.2678 - mean_squared_error: 10100.2678
Epoch 3/200
0s - loss: 8033.8670 - mean_squared_error: 8033.8670
Epoch 4/200
0s - loss: 5400.3572 - mean_squared_error: 5400.3572
Epoch 5/200
0s - loss: 2927.3481 - mean_squared_error: 2927.3481
Epoch 6/200
0s - loss: 1209.4728 - mean_squared_error: 1209.4728
Epoch 7/200
0s - loss: 356.1989 - mean_squared_error: 356.1989
Epoch 8/200
0s - loss: 71.3107 - mean_squared_error: 71.3107
Epoch 9/200
0s - loss: 10.1218 - mean_squared_error: 10.1218
Epoch 10/200
0s - loss: 1.1640 - mean_squared_error: 1.1640
Epoch 11/200
0s - loss: 0.2196 - mean_squared_error: 0.2196
Epoch 12/200
0s - loss: 0.1454 - mean_squared_error: 0.1454
Epoch 13/200
0s - loss: 0.1409 - mean_squared_error: 0.1409
Epoch 14/200
0s - loss: 0.1404 - mean_squared_error: 0.1404
Epoch 15/200
0s - loss: 0.1400 - mean_squared_error: 0.1400
Epoch 16/200
0s - loss: 0.1396 - mean_squared_error: 0.1396
E

0s - loss: 0.0100 - mean_squared_error: 0.0100
Epoch 140/200
0s - loss: 0.0094 - mean_squared_error: 0.0094
Epoch 141/200
0s - loss: 0.0088 - mean_squared_error: 0.0088
Epoch 142/200
0s - loss: 0.0082 - mean_squared_error: 0.0082
Epoch 143/200
0s - loss: 0.0077 - mean_squared_error: 0.0077
Epoch 144/200
0s - loss: 0.0072 - mean_squared_error: 0.0072
Epoch 145/200
0s - loss: 0.0067 - mean_squared_error: 0.0067
Epoch 146/200
0s - loss: 0.0062 - mean_squared_error: 0.0062
Epoch 147/200
0s - loss: 0.0058 - mean_squared_error: 0.0058
Epoch 148/200
0s - loss: 0.0054 - mean_squared_error: 0.0054
Epoch 149/200
0s - loss: 0.0050 - mean_squared_error: 0.0050
Epoch 150/200
0s - loss: 0.0046 - mean_squared_error: 0.0046
Epoch 151/200
0s - loss: 0.0042 - mean_squared_error: 0.0042
Epoch 152/200
0s - loss: 0.0039 - mean_squared_error: 0.0039
Epoch 153/200
0s - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 154/200
0s - loss: 0.0033 - mean_squared_error: 0.0033
Epoch 155/200
0s - loss: 0.0030 - mean

0s - loss: 0.1894 - mean_squared_error: 0.1894
Epoch 62/200
0s - loss: 0.1881 - mean_squared_error: 0.1881
Epoch 63/200
0s - loss: 0.1867 - mean_squared_error: 0.1867
Epoch 64/200
0s - loss: 0.1854 - mean_squared_error: 0.1854
Epoch 65/200
0s - loss: 0.1840 - mean_squared_error: 0.1840
Epoch 66/200
0s - loss: 0.1826 - mean_squared_error: 0.1826
Epoch 67/200
0s - loss: 0.1812 - mean_squared_error: 0.1812
Epoch 68/200
0s - loss: 0.1797 - mean_squared_error: 0.1797
Epoch 69/200
0s - loss: 0.1783 - mean_squared_error: 0.1783
Epoch 70/200
0s - loss: 0.1768 - mean_squared_error: 0.1768
Epoch 71/200
0s - loss: 0.1752 - mean_squared_error: 0.1752
Epoch 72/200
0s - loss: 0.1737 - mean_squared_error: 0.1737
Epoch 73/200
0s - loss: 0.1721 - mean_squared_error: 0.1721
Epoch 74/200
0s - loss: 0.1706 - mean_squared_error: 0.1706
Epoch 75/200
0s - loss: 0.1690 - mean_squared_error: 0.1690
Epoch 76/200
0s - loss: 0.1674 - mean_squared_error: 0.1674
Epoch 77/200
0s - loss: 0.1657 - mean_squared_error: 

0s - loss: 0.0097 - mean_squared_error: 0.0097
Epoch 200/200
0s - loss: 0.0097 - mean_squared_error: 0.0097
Baseline Error : 99.13478
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 5)                 15        
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 6         
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
#pickle.dump(acc_dict,open('acc_dict','wb'))
acc_dict = pickle.load(open('acc_dict','rb'))

In [13]:
'''
'Dishwasher' and 'Washing Machine' seem to be alright. Their accuracy remains similar.
But for the other attributes, it becomes negative at times. Why though? It should be something wrong with the way 
the data was interpolated(not using the graphical method) ?
On entire Dataset.

<->yr_data

'''
acc_dict

{0: [99.160624300870921,
  66.351393663699142,
  -1082488.7570248637,
  -40.475386170015668,
  51.007268550068979,
  99.97438543803311,
  99.183682942226682],
 1: [99.218887525078145,
  66.444416626776956,
  99.987170955871321,
  21.997674668950452,
  63.311279163806681,
  -899509.36595282692,
  99.126005713611363],
 2: [99.196965893899488,
  49.184226721768809,
  99.984742511079091,
  37.882776925176195,
  71.359430815175273,
  99.970936231610878,
  99.120346521962944],
 3: [99.232391909354561,
  68.965490871903356,
  99.992277716056449,
  -48.704974138119553,
  -16.361948893786987,
  99.94726629345044,
  99.160877056536165],
 4: [99.236996726531132,
  68.542564909509309,
  99.991007056799646,
  40.212594672401423,
  66.573431882178312,
  99.976283093380104,
  99.01197878491611]}

In [16]:
#pickle.dump(acc_dict1,open('acc_dict1','wb'))
acc_dict1 = pickle.load(open('acc_dict1','rb'))

In [17]:
'''
On the one-year dataset. hr_data
'''
acc_dict1

{0: [98.717805797071975,
  62.701512975793165,
  97.988387861320362,
  48.211459188754411,
  -485072510.67159933,
  98.361988383513335,
  98.381902832022661],
 1: [98.296165366020475,
  62.733447463524634,
  95.624056314931849,
  46.690713952194244,
  70.346646754661606,
  98.484823897779918,
  96.289621099117269],
 2: [98.799184559673222,
  59.957366922258586,
  98.824171050291085,
  47.849905168596287,
  70.398386954989832,
  99.511704499041898,
  96.010050556888004],
 3: [-480217.04307213111,
  61.041940420475889,
  98.839693204603137,
  48.859955590084724,
  68.924716673049318,
  98.340648546439638,
  -633193.00123723852],
 4: [99.035580515942087,
  60.288010892979315,
  97.944072893501072,
  48.91138439430862,
  70.890445207491069,
  99.137134069770866,
  96.042668884095306]}

In [ ]:
# plotting the dishwasher load for every 24hrs
i = 0
while(i <= x.shape[0]):
    s = np.array(read1.values[i:i+1439,0])
    k = [t[-9:-7]+t[-6:-4] for t in s ]
    k = [int(p) for p in k]
    l = read.values[i:i+1439,0]
    plt.grid(True)
    plt.plot(k,l)
    plt.show()
    i+=1440

In [32]:
d60 = pd.read_csv(path+"household_data_60min_singleindex.csv")
del d60['interpolated_values']
d60_cont1 = pd.DataFrame(d60.values[4893:8275])
d60_cont1.columns = d60.columns
del d60_cont1['cet_cest_timestamp']
print d60_cont1.shape

(3382, 10)


In [47]:
del d60_cont1['DE_KN_residential4_grid_export']
del d60_cont1['DE_KN_residential4_pv']
d60_cont1.head()

,utc_timestamp,DE_KN_residential4_dishwasher,DE_KN_residential4_ev,DE_KN_residential4_freezer,DE_KN_residential4_grid_import,DE_KN_residential4_heat_pump,DE_KN_residential4_refrigerator,DE_KN_residential4_washing_machine
0,2016-04-24T22:00:00Z,60.838,529.18,82.559,2911.01,2154.91,71.514,59.443
1,2016-04-24T23:00:00Z,60.838,529.18,82.573,2911.62,2155.22,71.55,59.443
2,2016-04-25T00:00:00Z,60.838,529.18,82.593,2912.24,2155.65,71.55,59.443
3,2016-04-25T01:00:00Z,60.838,529.18,82.613,2912.85,2156.09,71.585,59.443
4,2016-04-25T02:00:00Z,60.838,529.18,82.627,2913.43,2156.52,71.585,59.443


In [48]:
#Now try feeding this to the nn and see the results. This will help us in debugging what went wrong.
d60_cont1.isnull().sum(axis=0)

utc_timestamp                         0
DE_KN_residential4_dishwasher         0
DE_KN_residential4_ev                 0
DE_KN_residential4_freezer            0
DE_KN_residential4_grid_import        0
DE_KN_residential4_heat_pump          0
DE_KN_residential4_refrigerator       0
DE_KN_residential4_washing_machine    0
dtype: int64

In [70]:
#Testing the NN model on the continuous data without any interpolation.
acc_cont_dict={}

acc_cont_dict = nn_model(d60_cont1,acc_cont_dict,1)

print "The accuracies : ","\n\n",acc_cont_dict

The accuracies :  

{0: [99.268862800176677, 69.637953397437656, 98.253058016123859, -997456625.44378698, 98.517794783651482, 97.071639170660774, -635637.82446144591]}


In [2]:
os.getlogin()

'sfroot'